<a href="https://colab.research.google.com/github/Roterdamjr/ControleFinanceiro/blob/main/Controle_Financeiro_Python_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteca

> Adicionar aspas



In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 10.5 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
from io import BytesIO
import yfinance as yf

def load_data(excel_filepath):

    df_patrimonio = fn_busca_aba_excel(excel_filepath,'Patrimonio')
    df_alocacao_global = fn_busca_aba_excel(excel_filepath,'Alocacao global')
    df_cadastro_original_acoes = fn_busca_aba_excel(excel_filepath,'Cadastro Original Acoes')
    df_historico = fn_busca_aba_excel(excel_filepath,'Historico')
    df_cadastro_fi = fn_busca_aba_excel(excel_filepath,'Cadastro FI')
    df_cadastro_recebiveis = fn_busca_aba_excel(excel_filepath,'Cadastro Recebiveis')
    df_renda_fixa = fn_busca_aba_excel(excel_filepath,'Renda Fixa')
    df_exterior = fn_busca_aba_excel(excel_filepath,'Exterior')

    df_proporcoes_dh_dp =pd.read_excel(excel_filepath, sheet_name='Proporcoes DH_DP',  engine='openpyxl' )
    proporcao_dh = df_proporcoes_dh_dp['DH'].iloc[0] if not df_proporcoes_dh_dp['DH'].empty else 0.0
    proporcao_dp = df_proporcoes_dh_dp['DP'].iloc[0] if not df_proporcoes_dh_dp['DP'].empty else 0.0

    return (df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi,
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,
            proporcao_dh, proporcao_dp)

def fn_busca_aba_excel(excel_filepath,nome_aba ):
    df_aba = pd.read_excel(
        excel_filepath,
        sheet_name=nome_aba,
        engine='openpyxl'
    )

    df_aba = df_aba.dropna(axis=1,how='all')
    df_aba.set_index('Ativo', inplace=True)
    return df_aba

def fn_busca_pm_quantidade(df):

  #recebe dados de uma ativo e deveolve preco médio, quantidade

  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return qtde_total,preco_medio

def fn_calc_metricas(nome_ativo,
                     df_historico,
                     df_alocacao,
                     df_cotacoes ):

  ##############################
  # calcula quantidade, preco_medio, cotacao ,
  # patrimonio, resultado, rentabilidade, alocacao
  ##############################
  cotacao = df_cotacoes[nome_ativo]

  if nome_ativo in df_historico.index:
    df_ativo = df_historico.loc[nome_ativo]
    quantidade, preco_medio = fn_busca_pm_quantidade(df_ativo)

    alocacao = df_alocacao.loc[nome_ativo]['Alocacao']
    patrimonio = quantidade * cotacao
    resultado = patrimonio - quantidade * preco_medio
    rentabilidade = resultado / (quantidade * preco_medio)
  else:
      quantidade = 0
      preco_medio= 0
      alocacao = 0
      patrimonio = 0
      resultado = 0
      rentabilidade = 0

  return [int(quantidade),
          float(round(preco_medio, 2)),
          float(round(cotacao, 2)) ,
          float(round(patrimonio, 2)),
          float(round(resultado, 2)),
          float(round(rentabilidade, 2)),
          float(round(alocacao, 3))]

def fn_gera_carteira( p_df_historico, p_df_cadastro, p_df_cotacoes):
  df_carteira = pd.DataFrame([fn_calc_metricas(ativo, p_df_historico, p_df_cadastro, p_df_cotacoes)
                              for ativo in p_df_cadastro.index],
                          columns=['Quantidade', 'Preço Médio', 'Cotação', 'Patrimônio', 'Resultado', 'Rentabilidade', 'Alocacao'],
                          index=p_df_cadastro.index)

  # Totais
  patrimonio_total, resultado_total = df_carteira['Patrimônio'].sum(), df_carteira['Resultado'].sum()
  # Aportar
  #df_carteira['Aportar'] = ( df_carteira['Alocacao'] * (patrimonio_total + p_aporte) - df_carteira['Patrimônio'] )

  return df_carteira

def fn_busca_cotacoes(p_ativos):
  df_cotacoes = pd.DataFrame()
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
  df_cotacoes.index = df_cotacoes.index.str.replace('.SA', '', regex=False)
  return df_cotacoes

def fn_object_para_numerico(df,nome_coluna):

#    coluna_limpa = df[nome_coluna].astype(str).str.replace('.', '', regex=False)
#    coluna_limpa = coluna_limpa.str.replace(',', '.', regex=False)
 #   df[nome_coluna] = pd.to_numeric(coluna_limpa, errors='coerce')

    df[nome_coluna] = df[nome_coluna].str.replace('.', '', regex=False)
    df[nome_coluna] = df[nome_coluna].str.replace(',', '.', regex=False).astype(float)


def fn_percet_para_numerico(df, nome_coluna):

  coluna_limpa = (
          df[nome_coluna]
          .astype(str)
          .str.replace('%', '', regex=False)
          .str.replace(',', '.', regex=False)
      )

  df['Alocacao'] = pd.to_numeric(coluna_limpa, errors='coerce') / 100




In [84]:
import pandas as pd
import xlsxwriter

def fn_exporta(dataframes_dict, nome_arquivo):

    writer = pd.ExcelWriter(nome_arquivo, engine='xlsxwriter')
    workbook = writer.book

    # Definição de Cores e Formatos Base
    COR_CINZA = '#A9A9A9'
    COR_BEGE_CLARO = '#F5F5DC'
    COR_BEGE_PADRAO = '#F0EAD6'
    COR_VERDE = '#90EE90'
    BORDA = 1

    FORMATO_MOEDA = 'R$ #,##0.00'
    FORMATO_PERCENT = '0.00%'

    base_body_format = {'fg_color': COR_BEGE_PADRAO, 'border': BORDA}

    custom_formats_map = {
        'Ativo': (COR_BEGE_CLARO, None),
        'Patrimonio': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Preço Médio': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Cotação': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Resultado': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Aportar': (COR_VERDE, FORMATO_MOEDA),
        'Rentabilidade': (COR_BEGE_PADRAO, FORMATO_PERCENT),
        'Alocacao': (COR_BEGE_PADRAO, FORMATO_PERCENT),
    }

    formatos_excel = {}
    for col_name, (cor_fundo, num_format) in custom_formats_map.items():
        fmt_dict = {'fg_color': cor_fundo, 'border': BORDA}
        if num_format:
            fmt_dict['num_format'] = num_format
        formatos_excel[col_name] = workbook.add_format(fmt_dict)

    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': COR_CINZA,
        'border': BORDA
    })

    default_format = workbook.add_format(base_body_format)

    # Itera sobre cada DataFrame e Cria uma Nova Aba
    for sheet_name, df in dataframes_dict.items():

        df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=True)

        worksheet = writer.sheets[sheet_name]
        colunas_excel = [df.index.name or 'Ativo'] + list(df.columns.values)

        # Escreva o Cabeçalho e Defina a Largura das Colunas
        for col_num, value in enumerate(colunas_excel):
            worksheet.write(0, col_num, value, header_format)

            if value in ['Ativo', df.index.name]:
                width = 15
            elif value in ['Alocacao', 'Tipo']:
                width = 10
            elif value in ['Quantidade', 'Preço Médio', 'Cotação', 'Resultado']:
                width = 15
            else:
                width = 18

            worksheet.set_column(col_num, col_num, width)

        # Aplique a formatação no corpo da tabela
        num_linhas = len(df)
        for row in range(1, num_linhas + 1):

            for col_num, col_name in enumerate(colunas_excel):

                if col_name == (df.index.name or 'Ativo'):
                    valor = df.index[row - 1]
                else:
                    valor = df.iloc[row - 1][col_name]

                formato = formatos_excel.get(col_name, default_format)

                worksheet.write(row, col_num, valor, formato)

    # Salve o arquivo
    writer.close()

# Teste

In [ ]:
def fn_busca_cotacoes(p_ativos):
  df_cotacoes = pd.DataFrame()
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
  df_cotacoes.index = df_cotacoes.index.str.replace('.SA', '', regex=False)
  return df_cotacoes


df_cadastro_original_acoes, df_historico, df_cadastro_fi, df_cadastro_recebiveis, \
  proporcao_dh, proporcao_dp, aporte_dh, aporte_fii, aporte_recebiveis = load_data('/content/Controle Financeiro.xlsx' )

lista_de_ativos = df_cadastro_original_acoes.index.tolist()
lista_de_ativos.extend(df_cadastro_fi.index.tolist())
lista_de_ativos.extend(df_cadastro_recebiveis.index.tolist())
a = fn_busca_cotacoes(lista_de_ativos)
a


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

# Configuração do Ticker
TICKER = "VGIA11.SA"

# Busca a cotação do último dia útil
# Usamos um intervalo de 7 dias para garantir que pegamos o dia útil mais recente
data_fim = datetime.now().strftime('%Y-%m-%d')
data_inicio = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

dados = yf.download(TICKER, start=data_inicio, end=data_fim)

if not dados.empty:
    # Seleciona a última linha (data mais recente)
    ultima_linha = dados.iloc[-1]

    # Extrai a data e o preço de fechamento
    data_recente = ultima_linha.name.strftime('%Y-%m-%d')
    preco_fechamento = ultima_linha['Close']

    print(preco_fechamento)
else:
    print(f"Nenhum dado encontrado para o ticker {TICKER}.")



/tmp/ipython-input-1255517619.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dados = yf.download(TICKER, start=data_inicio, end=data_fim)
[*********************100%***********************]  1 of 1 completed

Ticker
VGIA11.SA    9.71
Name: 2025-12-01 00:00:00, dtype: float64


# Planilha e cotacoes

In [78]:
####################################
#     Download da Planilha
####################################
df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi, \
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,\
            proporcao_dh, proporcao_dp = load_data('/content/Controle Financeiro.xlsx' )

In [79]:
####################################
#     Download de Cotações
####################################
lista_de_ativos = df_cadastro_original_acoes.index.tolist()
lista_de_ativos.extend(df_cadastro_fi.index.tolist())
lista_de_ativos.extend(df_cadastro_recebiveis.index.tolist())
df_cotacoes = fn_busca_cotacoes(lista_de_ativos)

/tmp/ipython-input-2358342415.py:111: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
[*********************100%***********************]  32 of 32 completed


#  Cadastro ponderado de acoes, FI e Recebiveis

In [81]:
################################################################
# Geracao de df_cadastro_acoes_ponderado
# que se baeia nos pesos do Dica Acoes e Dica Prev
################################################################
df_acoes_dh = df_cadastro_original_acoes.loc[df_cadastro_original_acoes['Tipo'] == 'DH'].copy()
soma_total = df_acoes_dh['Alocacao'].sum()
df_acoes_dh['Calculado'] = df_acoes_dh['Alocacao'] / soma_total * proporcao_dh

df_acoes_dp = df_cadastro_original_acoes.loc[df_cadastro_original_acoes['Tipo'] == 'DP'].copy()
soma_total = df_acoes_dp['Alocacao'].sum()
df_acoes_dp['Calculado'] = df_acoes_dp['Alocacao'] / soma_total * proporcao_dp

df_cadastro_acoes_ponderado = pd.concat([df_acoes_dh, df_acoes_dp]).sort_index()
df_cadastro_acoes_ponderado.drop('Alocacao', axis=1, inplace=True)
df_cadastro_acoes_ponderado.rename(columns={'Calculado': 'Alocacao'}, inplace=True)
df_cadastro_acoes_ponderado['Alocacao'] = df_cadastro_acoes_ponderado['Alocacao'].round(3)

####################################
#  Geração das carteiras de Ações,
#  Fundos imobiliarios e Recebiveis
####################################
df_carteira_acoes = fn_gera_carteira  (df_historico, df_cadastro_acoes_ponderado, df_cotacoes)
df_carteira_acoes['Preco Justo'] = df_cadastro_acoes_ponderado['Preco Justo']

df_carteira_fi = fn_gera_carteira  (df_historico, df_cadastro_fi, df_cotacoes)
df_carteira_recebiveis = fn_gera_carteira  (df_historico, df_cadastro_recebiveis, df_cotacoes)


# Carteira Estruturada Global

In [82]:
df_carteira_global  = pd.DataFrame(columns=['Ativo', 'Patrimonio', 'Alocacao', 'Aportar']).set_index('Ativo')

linhas_a_atualizar = ['Acoes', 'Fundos Imobiliarios','Recebiveis','CDB Dan','CDB Marcia',
                      'Pre-Fixado','Selic','IPCA', 'VGBL Dan','VGBL Marcia', 'Exterior', 'Opcoes']

df_carteira_global = df_carteira_global.reindex(df_carteira_global.index.union(linhas_a_atualizar))
df_carteira_global.loc[linhas_a_atualizar, 'Patrimonio'] = df_patrimonio['Patrimonio'].loc[linhas_a_atualizar]

#### Alocação ###
df_carteira_global.loc['Acoes', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Acoes']
df_carteira_global.loc['Fundos Imobiliarios', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['FII']
df_carteira_global.loc['Recebiveis', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Recebiveis']

fator_base = df_alocacao_global.loc['Renda Fixa', 'Alocacao']
indices_rf = df_renda_fixa.index
df_carteira_global.loc[indices_rf, 'Alocacao'] = (
    round(fator_base * df_renda_fixa['Alocacao'],2)
)

df_carteira_global.loc['Exterior', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Exterior']
df_carteira_global.loc['Opcoes', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Opcoes']

df_carteira_global['Aportar'] = df_carteira_global['Alocacao'] * df_carteira_global['Patrimonio'].sum() - df_carteira_global['Patrimonio']

###### calcula e atualiza coluna Aportar        #####
###### em na carteira de Acoes, FI e Recebiveis #####
tot = df_carteira_acoes['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Acoes']
df_carteira_acoes['Aportar'] =  df_carteira_acoes['Alocacao'] * (tot + aporte) - df_carteira_acoes['Patrimônio']

tot = df_carteira_fi['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Fundos Imobiliarios']
df_carteira_acoes['Aportar'] =  df_carteira_acoes['Alocacao'] * (tot + aporte) - df_carteira_acoes['Patrimônio']

tot = df_carteira_recebiveis['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Recebiveis']
df_carteira_acoes['Aportar'] =  df_carteira_acoes['Alocacao'] * (tot + aporte) - df_carteira_acoes['Patrimônio']


In [77]:
df_carteira_acoes

,Quantidade,Preço Médio,Cotação,Patrimônio,Resultado,Rentabilidade,Alocacao,Preco Justo,Aportar
Ativo,,,,,,,,,
ABCB4,670,19.94,25.00,16750.00,3387.17,0.25,0.080,25.00,7902.87280
ALOS3,430,22.07,28.98,12461.40,2972.43,0.31,0.045,27.00,1405.84095
BBAS3,480,26.26,22.51,10806.80,-1797.39,-0.14,0.000,20.00,-10806.80000
BBDC4,900,13.44,19.54,17586.00,5494.00,0.45,0.106,20.00,15079.05646
CSMG3,300,22.01,43.61,13083.00,6481.29,0.98,0.080,25.00,11569.87280
CXSE3,1300,14.10,16.14,20982.00,2657.46,0.15,0.106,17.50,11683.05646
EGIE3,90,41.16,31.07,2796.30,-907.98,-0.25,0.023,42.00,4291.40093
FLRY3,300,12.95,15.87,4759.54,875.54,0.23,0.023,14.50,2328.16093
ITSA4,1636,9.95,12.54,20515.44,4244.29,0.26,0.085,11.00,5678.23735


# Exportação

In [87]:
# Crie um dicionário com todos os DataFrames e os nomes desejados para as abas
# Substitua df_carteira_acoes, df_carteira_fi, etc. pelos seus DataFrames reais.
dfs_to_export = {
    'Global': df_carteira_global,
    'Ações': df_carteira_acoes,
    'FII': df_carteira_fi,
    'Recebíveis': df_carteira_recebiveis
}

# Exporta todos para um único arquivo
fn_exporta(dfs_to_export, 'Carteira Global de Investimentos.xlsx')